In [2]:
from bs4 import BeautifulSoup

# Open and read the SVG file
with open('2782_3.svg', 'r') as file:
    svg_content = file.read()

# Parse the SVG content using BeautifulSoup with lxml's XML parser
soup = BeautifulSoup(svg_content, 'xml')
min_length = 3000
max_length = 6000

selected_paths = []

for path in soup.find_all('path'):
    d_length = len(path.get('d', ''))  # Get the length of the `d` attribute
    if min_length <= d_length <= max_length:
        selected_paths.append(path)

# Output the paths or manipulate them as needed
for path in selected_paths:
    print(path)
    print("Length of 'd' attribute:", len(path['d']))
    
    # Traverse up the tree to print parent elements and their attributes
    parent = path.parent
    while parent:
        print("Parent Element:", parent.name)
        for attribute, value in parent.attrs.items():
            print(f"{attribute}: {value}")
        parent = parent.parent

<svg:path d="m 0,0 h 0.96 c 0.48,-0.96 0.48,-1.44 0.96,-1.44 0.48,-0.48 0.48,-0.48 0.96,-0.48 0.96,0 1.44,0 1.92,0.48 0,0.48 0.48,0.96 0.48,1.44 0,0.48 -0.48,0.96 -0.48,1.44 -0.48,0.48 -0.96,0.48 -1.44,0.48 -0.48,0 -0.48,0 -0.96,0 v 0.96 c 0,0 0,0 0.48,0 0.48,0 0.96,0.48 1.44,0.48 0,0.48 0.48,0.96 0.48,1.44 0,0.48 0,0.96 -0.48,0.96 C 3.84,6.24 3.36,6.24 2.88,6.24 2.4,6.24 2.4,6.24 1.92,5.76 1.44,5.76 1.44,5.28 1.44,4.32 L 0,4.8 C 0,5.28 0.48,6.24 0.96,6.72 1.44,7.2 2.4,7.2 2.88,7.2 c 0.48,0 1.44,0 1.92,-0.48 0,0 0.48,-0.48 0.96,-0.96 0,0 0,-0.48 0,-0.96 0,-0.48 0,-0.96 0,-1.44 C 5.28,3.36 5.28,2.88 4.8,2.4 5.28,2.4 5.76,2.4 5.76,1.92 6.24,1.44 6.72,0.96 6.72,0 6.72,-0.96 6.24,-1.44 5.76,-1.92 4.8,-2.88 3.84,-2.88 2.88,-2.88 2.4,-2.88 1.44,-2.88 0.96,-2.4 0.48,-1.92 0,-0.96 0,0 Z m 11.52,-2.88 v 2.4 H 7.2 V 0.48 L 12,7.2 h 0.96 V 0.48 h 1.44 v -0.96 h -1.44 v -2.4 z m 0,3.36 v 4.8 L 8.64,0.48 Z m 7.2,-3.36 V 6.24 H 15.36 V 7.2 h 8.16 V 6.24 h -3.36 v -9.12 z m 5.76,0 V 7.2 h 3.84 c 0.96

In [ ]:
In this code: "from bs4 import BeautifulSoup

# Open and read the SVG file
with open('2681_2.svg', 'r') as file:
    svg_content = file.read()

# Parse the SVG content using BeautifulSoup with lxml's XML parser
soup = BeautifulSoup(svg_content, 'xml')
min_length = 3000
max_length = 6000

# List to hold paths that meet criteria
selected_paths = []

for path in soup.find_all('path'):
    d_length = len(path.get('d', ''))  # Get the length of the `d` attribute
    if min_length <= d_length <= max_length:
        selected_paths.append(path)

# Output the paths or manipulate them as needed
for path in selected_paths:
    print(path)
    print("Length of 'd' attribute:", len(path['d']))

" how can i additionally print all parent elements of the found elements. But from the parents i just want to print there attributes and the corresponding values

## find all used colors

In [25]:
from bs4 import BeautifulSoup

# Open and read the SVG file
with open('2681_2.svg', 'r') as file:
    svg_content = file.read()

soup = BeautifulSoup(svg_content, 'xml')  # Use 'xml' parser for parsing SVG


unique_colors = set()  # Set to store unique colors

for path in soup.find_all('path'):
    style = path.get('style')  # Get the style attribute of the path
    if style:
        styles = style.split(';')  # Split style into individual properties
        fill_style = [s for s in styles if 'fill:' in s]  # Find the fill style
        if fill_style:
            color = fill_style[0].split(':')[1].strip()  # Extract the color value
            unique_colors.add(color)  # Add to set of unique colors

# Output the unique colors
print(unique_colors)

{'#00ffff', '#00ff00', 'none', '#ff0000', '#000000', '#ffffff', '#ffff00'}


## find yellow cross element

In [26]:

soup = BeautifulSoup(svg_content, 'xml')

target_color = '#ffff00'
paths_with_target_color = []

for path in soup.find_all('path'):
    # Check the fill attribute directly
    if path.get('fill') == target_color:
        paths_with_target_color.append(path)

    # Check within the style attribute
    elif path.get('style'):
        styles = dict(item.split(':') for item in path['style'].split(';') if item)
        if 'fill' in styles and styles['fill'].strip() == target_color:
            paths_with_target_color.append(path)

# Output the paths or manipulate them as needed
for path in paths_with_target_color:
    print(path)

<svg:path d="m 0,0 -5.76,-0.96 1.44,-2.4 -2.88,0.96 -1.92,-1.44 v 1.92 h -1.92 L -9.6,0 -12,1.44 -9.12,1.92 -8.64,3.84 -7.68,2.4 -4.32,5.76 -5.76,1.92 Z" id="path168" style="fill:#ffff00;fill-opacity:1;fill-rule:nonzero;stroke:none"/>
<svg:path d="m 0,0 -5.76,-0.96 1.44,-2.4 -2.88,0.96 -1.92,-1.44 v 1.92 h -1.92 L -9.6,0 -12,1.44 -9.12,1.92 -8.64,3.84 -7.68,2.4 -4.32,5.76 -5.76,1.92 Z" id="path180" style="fill:#ffff00;fill-opacity:1;fill-rule:nonzero;stroke:none"/>
<svg:path d="m 0,0 -5.76,-0.96 1.44,-2.4 -2.88,0.96 -1.92,-1.44 v 1.92 h -1.92 L -9.6,0 -12,1.44 -9.12,1.92 -8.64,3.84 -7.68,2.4 -4.32,5.76 -5.76,1.92 Z" id="path192" style="fill:#ffff00;fill-opacity:1;fill-rule:nonzero;stroke:none"/>
<svg:path d="m 0,0 -5.76,-0.96 1.44,-2.4 -2.88,0.96 -1.92,-1.44 v 1.92 h -1.92 L -9.6,0 -12,1.44 -9.12,1.92 -8.64,3.84 -7.68,2.4 -4.32,5.76 -5.76,1.92 Z" id="path204" style="fill:#ffff00;fill-opacity:1;fill-rule:nonzero;stroke:none"/>
<svg:path d="m 0,0 -5.76,-0.96 1.44,-2.4 -2.88,0.96 -1.92,-1

In [ ]:
import cairosvg

# Function to convert SVG to PNG
def svg_to_png(svg_input_path, png_output_path):
    cairosvg.svg2png(url=svg_input_path, write_to=png_output_path)

# Example usage
svg_to_png("input.svg", "output.png")